In [ ]:
from ranx import Qrels, Run

In [ ]:
qrels = Qrels.from_ir_datasets("beir/scifact/test")

In [ ]:
print(type(qrels))

In [ ]:
from rerankers import Reranker

In [ ]:
import srsly

# corpus = [x for x in srsly.read_jsonl('data/wikipedia/data/collections/docs00.json')]
queries = [x for x in srsly.read_json('data/msmarco/query/default_query.json')]

# print(corpus[0])
print(queries[0])
print(len(queries))

In [ ]:
ranker = Reranker('castorini/monot5-base-msmarco-10k', device='cuda', batch_size=128, verbose=0)

In [ ]:
top100 = srsly.read_json('data/scifact/scifact_top_100.json')



In [ ]:
corpus_map = {x['_id']: f"{x['title']} {x['text']}" for x in corpus}

In [ ]:
qrels_dict = dict(qrels)
queries = [q for q in queries if q['_id'] in qrels_dict]
from tqdm import tqdm

scores = {}
for q in tqdm(queries):
    doc_ids = top100[q['_id']]
    docs = [corpus_map[x] for x in doc_ids]
    scores[q['_id']] = ranker.rank(q['text'], docs, doc_ids=doc_ids)


In [ ]:
scores_dict = {}
for q_id, ranked_results in scores.items():
    top_10_results = ranked_results.top_k(10)
    scores_dict[q_id] = {result.doc_id: result.score for result in top_10_results}
run = Run(scores_dict)

In [ ]:
from ranx import evaluate
evaluation_score = evaluate(qrels, run, 'ndcg@10')
litterature_result = 0.734 # From RankGPT Paper https://arxiv.org/pdf/2304.09542.pdf
if abs(evaluation_score - litterature_result) > 0.01:
    print(f"Score {evaluation_score:0.3f} differs by more than 0.01 from the the reported score.")
else:
    print(f"Score is within 0.01 NDCG@10 of the reported score!")
